# PREDICTION

In [1]:
from offerclassif.data import load_json, get_df
from offerclassif.embedding import get_vectors
from offerclassif.utils import plot_tsne, topNoob
from sklearn.ensemble import ExtraTreesClassifier
import plotly.express as px
import numpy as np

# 💾 Chargement des données
---

In [2]:
thesaurus = load_json('../data/external/job_thesaurus.json')
train = load_json('../data/external/offers_train.json')

df = get_df(train, thesaurus)

20 items loaded from ../data/external/job_thesaurus.json.
84 items loaded from ../data/external/offers_train.json.


# ⚗️ Prédiction du libellé
---

In [3]:
# Récupération de la représentation vectorielle
X = get_vectors(df.title)

In [4]:
# Apprentissage du modèle
et = ExtraTreesClassifier(oob_score=True, bootstrap=True, n_estimators=1000, random_state=1) 
et.fit(X, df.internal_label)


ExtraTreesClassifier(bootstrap=True, n_estimators=1000, oob_score=True,
                     random_state=1)

In [5]:
# Accuracy
print("Accuracy: {:.1%}".format(et.oob_score_))
print("Top 3 accuracy: {:.1%}".format(topNoob(et, df.internal_label, 3)))
print("Top 2 accuracy: {:.1%}".format(topNoob(et, df.internal_label, 2)))

Accuracy: 75.0%
Top 3 accuracy: 91.7%
Top 2 accuracy: 84.5%


Trois chances sur quatre d'avoir le bon libellé ... 🎉 et plus de 9 chances sur 10 d'avoir le bon libellé dans le top 3 des libellés prédits !

In [6]:
df['pred'] = et.oob_decision_function_.argmax(axis=1)
df['pred'] = df.pred.apply(lambda x: et.classes_[x])

## Matrice de confusion

In [7]:
print("<real_label> (n=)")
print("   <predicted_label> (n=)")
print("")

for label in df.internal_label.unique():
    sub_df = df.query(f"internal_label == '{label}'")
    print(f"{label} (n={len(sub_df)})")
    counts = sub_df.pred.value_counts()
    for i,j in counts.iteritems():
        print(f"   {i} (n={j})")
    print("")

<real_label> (n=)
   <predicted_label> (n=)

Vendeur télephonie mobile (n=1)
   Vendeur (n=1)

Ingénieur de recherche (n=9)
   Ingénieur de recherche (n=8)
   Data Scientist (n=1)

Data Scientist (n=11)
   Data Scientist (n=8)
   Ingénieur de recherche (n=2)
   Vendeur (n=1)

Ingénieur calcul mécanique (n=5)
   Ingénieur calcul mécanique (n=2)
   Ingénieur calcul (n=2)
   Data Scientist (n=1)

Vendeur (n=18)
   Vendeur (n=18)

Ingénieur calcul (n=4)
   Ingénieur calcul mécanique (n=3)
   Ingénieur calcul (n=1)

Développeur Python (n=6)
   Développeur C++ (n=4)
   Développeur Python (n=2)

Chargé de clientele (n=9)
   Chargé de clientele (n=9)

Développeur C++ (n=8)
   Développeur C++ (n=6)
   Développeur Python (n=2)

Commercial (n=6)
   Commercial (n=5)
   Vendeur (n=1)

Développeur Java (n=6)
   Développeur Java (n=4)
   Développeur C++ (n=2)

Consultant Java (n=1)
   Développeur Java (n=1)



#  ⚗️ Prediction du secteur 
---

In [8]:
et_sect = ExtraTreesClassifier(oob_score=True, bootstrap=True, n_estimators=1000, random_state=1) 
et_sect.fit(X, df.sector_internal_label)

ExtraTreesClassifier(bootstrap=True, n_estimators=1000, oob_score=True,
                     random_state=1)

In [9]:
# Accuracy
print("Accuracy: {:.1%}".format(et_sect.oob_score_))

Accuracy: 96.4%


96% de chance de prédire le bon secteur !

## Prevision du libellé en prenant en compte les résultats du modèle de secteur

In [10]:
new_X = np.hstack([X, et_sect.oob_decision_function_])

In [11]:
et_lib = ExtraTreesClassifier(oob_score=True, bootstrap=True, n_estimators=1000, random_state=1) 
et_lib.fit(new_X, df.internal_label)

ExtraTreesClassifier(bootstrap=True, n_estimators=1000, oob_score=True,
                     random_state=1)

In [12]:
# Accuracy
print("Accuracy: {:.1%}".format(et_lib.oob_score_))
print("Top 3 accuracy: {:.1%}".format(topNoob(et_lib, df.internal_label, 3)))
print("Top 2 accuracy: {:.1%}".format(topNoob(et_lib, df.internal_label, 2)))

Accuracy: 77.4%
Top 3 accuracy: 91.7%
Top 2 accuracy: 85.7%


Ajouter une information sur le secteur permet d'améliorer sensiblement la prédiction du libellé 

In [13]:
df['pred_2'] = et_lib.oob_decision_function_.argmax(axis=1)
df['pred_2'] = df.pred_2.apply(lambda x: et_lib.classes_[x])

## Matrice de confusion

In [14]:
print("<real_label> (n=)")
print("   <predicted_label> (n=)")
print("")

for label in df.internal_label.unique():
    sub_df = df.query(f"internal_label == '{label}'")
    print(f"{label} (n={len(sub_df)})")
    counts = sub_df.pred_2.value_counts()
    for i,j in counts.iteritems():
        print(f"   {i} (n={j})")
    print("")

<real_label> (n=)
   <predicted_label> (n=)

Vendeur télephonie mobile (n=1)
   Vendeur (n=1)

Ingénieur de recherche (n=9)
   Ingénieur de recherche (n=8)
   Data Scientist (n=1)

Data Scientist (n=11)
   Data Scientist (n=8)
   Ingénieur de recherche (n=2)
   Vendeur (n=1)

Ingénieur calcul mécanique (n=5)
   Ingénieur calcul mécanique (n=3)
   Ingénieur calcul (n=2)

Vendeur (n=18)
   Vendeur (n=18)

Ingénieur calcul (n=4)
   Ingénieur calcul mécanique (n=3)
   Ingénieur calcul (n=1)

Développeur Python (n=6)
   Développeur C++ (n=3)
   Développeur Python (n=2)
   Ingénieur de recherche (n=1)

Chargé de clientele (n=9)
   Chargé de clientele (n=9)

Développeur C++ (n=8)
   Développeur C++ (n=6)
   Développeur Python (n=2)

Commercial (n=6)
   Commercial (n=6)

Développeur Java (n=6)
   Développeur Java (n=4)
   Développeur C++ (n=2)

Consultant Java (n=1)
   Développeur Java (n=1)

